In [1]:
import gmsh, pygmsh
import capytaine as cpy
import autograd.numpy as np
from autograd import grad, jacobian
import matplotlib.pyplot as plt
import xarray as xr
from scipy.optimize import brute

import wecopttool as wot

## set colorblind-friendly colormap for plots
plt.style.use('tableau-colorblind10')

In [2]:
f1 = 0.02
nfreq = 50

filename = './data/bem.nc'
bem_data = wot.read_netcdf(filename)

In [3]:
waves = None 
ndof = 4
wec = wot.WEC.from_bem(bem_data,
                       inertia_matrix=np.eye(ndof),
                       hydrostatic_stiffness=np.eye(ndof),
                       constraints=None,
                       friction=None,
                       f_add=None,
                       dof_names=bem_data.influenced_dof.values,
)

In [4]:
x_wec = np.ones(400)
x_opt = np.ones(100)

In [5]:
nsubsteps = 1

sprocket_diameter = 0.13
sprocket_MOI = 0.0060

idler_pulley_diameter = 4.25
idler_pulley_MOI = 0.0050

generator_MOI = 1.786e-2
generator_winding_resistance = 5.87
generator_winding_inductance = 0.0536
generator_torque_constant = 8.51

drivetrain_friction = 0.5
drivetrain_stiffness = 0.0

pto_ndof = 1
kinematics = np.array([[1, -1, 0, 0],])
controller = None
loss = None
name = ['PTO_Heave',]

def gear_ratio(pulley_radius):
    return 1/pulley_radius  # rad/m 

def fun1(sprocket_diameter):
    # omega=bem_data.omega.values
    # pulley_ratio = sprocket_diameter / idler_pulley_diameter
    # drivetrain_inertia = (
    #     generator_MOI + 
    #     sprocket_MOI + 
    #     2 * idler_pulley_MOI*pulley_ratio**2
    # )  # N*m^2
    # drivetrain_impedance = (
    #     1j*omega*drivetrain_inertia + 
    #     drivetrain_friction + 
    #     1/(1j*omega)*drivetrain_stiffness
    # ) 
    # winding_impedance = (generator_winding_resistance
    #                    + 1j*omega*generator_winding_inductance
    # )
    # pulley_radius = sprocket_diameter / 2
    # pto_impedance_11 = -1* gear_ratio(pulley_radius)**2 * drivetrain_impedance
    # off_diag = -1*np.ones(omega.shape) * (
    #     np.sqrt(3.0/2.0) * generator_torque_constant * gear_ratio(pulley_radius) + 0j)
    # pto_impedance_12 = off_diag 
    # pto_impedance_21 = off_diag
    # pto_impedance_22 = winding_impedance
    # pto_impedance = np.array([[pto_impedance_11, pto_impedance_12],
    #                       [pto_impedance_21, pto_impedance_22]])
    
    pto_impedance = np.ones([2, 2, nfreq])*sprocket_diameter + 0j

    pto = wot.pto.PTO(pto_ndof, kinematics, controller, pto_impedance, loss, name)
    return np.sum(pto.transfer_mat)
    # return pto.average_power(wec, x_wec, x_opt, waves, nsubsteps)

grad1 = grad(fun1, 0)
jac1 = jacobian(fun1, 0)

In [6]:
fun1(0.13)

769.230769230769

In [7]:
grad1(0.13)

-5917.15976331361